In [247]:
import pandas as pd
import numpy as np

data = pd.read_csv("Sales_Transactions_Dataset_Weekly.csv")
data = data.filter(regex="Product|W").copy()

data['Product_Code_NUM'] = data['Product_Code'].str.extract("(\d+)").astype(int)

data.head()

,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,W43,W44,W45,W46,W47,W48,W49,W50,W51,Product_Code_NUM
0,P1,11,12,10,8,13,12,14,21,6,...,7,8,10,12,3,7,6,5,10,1
1,P2,7,6,3,2,7,1,6,3,3,...,4,5,1,1,4,5,1,6,0,2
2,P3,7,11,8,9,10,8,7,13,12,...,14,5,5,7,8,14,8,8,7,3
3,P4,12,8,13,5,9,6,9,13,13,...,10,3,4,6,8,14,8,7,8,4
4,P5,8,5,13,11,6,7,9,14,9,...,11,7,12,6,6,5,11,8,9,5


In [248]:
print("Max Product_Code: {} - Unique Product_Code: {}".format(data['Product_Code_NUM'].max(), data['Product_Code_NUM'].nunique()))

Max Product_Code: 819 - Unique Product_Code: 811


In [249]:
X_train = []
Y_train = []

X_test = []
Y_test = []

for ix, row in data.iterrows():
    for w in range(8, 50):
        product_code_num = row['Product_Code_NUM']
        x = row.iloc[w-7:w].values.astype(int)
        y = row.iloc[w:w+3].values.astype(int)

        product_code_num_ohe = to_categorical(product_code_num, num_classes=820)
        x = np.append(x, product_code_num_ohe)

        if w < 30:
            X_train.append(x)
            Y_train.append(y)
        else:
            X_test.append(x)
            Y_test.append(y)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_test = np.array(X_test)
Y_test = np.array(Y_test)
    

In [250]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((17842, 827), (16220, 827), (17842, 3), (16220, 3))

In [251]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [252]:
from keras import Model
from keras.layers import Dense, Input, Dropout

inp = Input(shape=(827,))
hid1 = Dense(64, activation='relu')(inp)
out  = Dense(3, activation='linear')(hid1)

mdl = Model(inputs=inp, outputs=out)
mdl.compile(loss='mse', optimizer='adam')

mdl.fit(X_train,Y_train, shuffle=True, validation_data=[X_test, Y_test], epochs=20, batch_size=32)

Train on 17842 samples, validate on 16220 samples
Epoch 1/20
17842/17842 [==============================] - 1s 67us/step - loss: 52.9011 - val_loss: 12.6848
Epoch 2/20
17842/17842 [==============================] - 1s 48us/step - loss: 15.7175 - val_loss: 12.1521
Epoch 3/20
17842/17842 [==============================] - 1s 46us/step - loss: 15.6286 - val_loss: 12.0049
Epoch 4/20
17842/17842 [==============================] - 1s 47us/step - loss: 15.5743 - val_loss: 12.2688
Epoch 5/20
17842/17842 [==============================] - 1s 47us/step - loss: 15.5218 - val_loss: 12.7508
Epoch 6/20
17842/17842 [==============================] - 1s 49us/step - loss: 15.5067 - val_loss: 12.0232
Epoch 7/20
17842/17842 [==============================] - 1s 46us/step - loss: 15.4836 - val_loss: 12.3858
Epoch 8/20
17842/17842 [==============================] - 1s 48us/step - loss: 15.4385 - val_loss: 12.8957
Epoch 9/20
17842/17842 [==============================] - 1s 46us/step - loss: 15.3570 - val_l

In [253]:
p = mdl.predict(X_test)

np.sqrt(((np.log1p(p) - np.log1p(Y_test))**2)).mean(axis=0)

array([0.29417287, 0.29790673, 0.30176874])